# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902026


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/39 [00:00<?, ?it/s]

Rendering models:   5%|▌         | 2/39 [00:01<00:33,  1.12it/s]

Rendering models:   8%|▊         | 3/39 [00:07<01:19,  2.20s/it]

Rendering models:  10%|█         | 4/39 [00:07<00:55,  1.59s/it]

Rendering models:  13%|█▎        | 5/39 [00:07<00:38,  1.15s/it]

...

Rendering models:  85%|████████▍ | 33/39 [00:10<00:00,  8.70it/s]

Rendering models:  87%|████████▋ | 34/39 [00:10<00:00,  8.65it/s]

Rendering models:  92%|█████████▏| 36/39 [00:10<00:00,  7.73it/s]

Rendering models:  97%|█████████▋| 38/39 [00:10<00:00,  9.06it/s]

equidad1                              0.000898
not-logged-in-cae133427d99aac1e554    0.000688
Khaazel                               0.000766
Huskynator                            0.000707
kayleebug2017                         0.001712
not-logged-in-8c0c44faf754d344b18a    0.009286
not-logged-in-9f66093d0c2b8fe22169    0.001405
not-logged-in-71f9718462c58a56588f    0.001039
not-logged-in-8c0c44faf754d344b18a    0.038611
Thedunkmasta                          0.003079
Thedunkmasta                          0.003077
Thedunkmasta                          0.002806
Thedunkmasta                          0.004327
Thedunkmasta                          0.002550
Thedunkmasta                          0.003553
Thedunkmasta                          0.003764
Thedunkmasta                          0.003062
Thedunkmasta                          0.002766
not-logged-in-846e14e9f3e0f594a568    0.004785
Synapsestar                           0.000450
not-logged-in-314d48142a3a122ba666    0.002555
crush202020  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())